In [31]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("HIP version:", torch.version.hip)
print("Device count:", torch.cuda.device_count())
print("Device 0:", torch.cuda.get_device_name(0))

Using device: cuda
HIP version: 6.2.41133-dd7f95766
Device count: 1
Device 0: AMD Radeon RX 6800


In [32]:
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

In [33]:
transform_train = transforms.Compose([
    transforms.AutoAugment(transforms.AutoAugmentPolicy.CIFAR10),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [34]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

val_size = 5000
test_size = len(test_data) - val_size

val_data, test_data = random_split(test_data, [val_size, test_size])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print(f'Training dataset: {len(train_loader.dataset)}')
print(f'Validation dataset: {len(val_loader.dataset)}')
print(f'Test dataset: {len(test_loader.dataset)}')


Files already downloaded and verified
Files already downloaded and verified
Training dataset: 50000
Validation dataset: 5000
Test dataset: 5000


In [35]:
image, label = train_data[0]

In [36]:
image.size()

torch.Size([3, 32, 32])

In [37]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [38]:
class MyResnet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.backbone = resnet18(weights=None)
        self.backbone.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone.maxpool = nn.Identity()
        self.backbone.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        return self.backbone(x)


In [39]:
net = MyResnet()
net = net.to(device)

In [40]:
#net = NeuralNet().to(device)
loss_function = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler(device)

num_of_epochs = 70

optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_of_epochs)

torch.manual_seed(42)

In [41]:
writer = SummaryWriter()

In [42]:

highest_val_accuracy = 0.0

for epoch in range(num_of_epochs):
    net.train()

    running_loss = 0.0
    for _, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = net(inputs)
            loss = loss_function(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    net.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    if val_accuracy > highest_val_accuracy:
        highest_val_accuracy = val_accuracy

        checkpoint = {
            "epoch": epoch,
            "model_state": net.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "scheduler_state": scheduler.state_dict(),
            "scaler_state": scaler.state_dict(),
            "config": {
                "conv1_kernel": 3,
                "remove_maxpool": True,
                "num_classes": 10
            },
            "best_val_accuracy": highest_val_accuracy
        }

        torch.save(checkpoint, "trained_model.pth")
    
    global_step = epoch + 1
    writer.add_scalars("Loss", {'Train': avg_train_loss, 'Val': avg_val_loss}, global_step)
    writer.add_scalar("Loss/Train", avg_train_loss, global_step)
    writer.add_scalar("Loss/Val", avg_val_loss, global_step)
    writer.add_scalar("Accuracy/Val", val_accuracy, global_step)
    writer.add_scalar("Params/LR", optimizer.param_groups[0]["lr"], global_step)

    print(
        f'Epoch ({epoch + 1}/{num_of_epochs}),'
        f'Train Loss: {avg_train_loss:.4f}, '
        f'Val Loss: {avg_val_loss:.4f}, '
        f'Val Accuracy: {val_accuracy:.2f}%'
        )

    scheduler.step()

writer.flush()

print(f'Highest Validation Accuracy: {highest_val_accuracy:.2f}%')

Epoch (1/70),Train Loss: 2.0520, Val Loss: 1.8200, Val Accuracy: 33.60%
Epoch (2/70),Train Loss: 1.5546, Val Loss: 1.5124, Val Accuracy: 49.58%
Epoch (3/70),Train Loss: 1.3093, Val Loss: 1.1284, Val Accuracy: 59.82%
Epoch (4/70),Train Loss: 1.1832, Val Loss: 0.9630, Val Accuracy: 66.52%
Epoch (5/70),Train Loss: 1.1099, Val Loss: 0.9740, Val Accuracy: 65.80%
Epoch (6/70),Train Loss: 1.0671, Val Loss: 0.9897, Val Accuracy: 65.92%
Epoch (7/70),Train Loss: 1.0412, Val Loss: 1.2637, Val Accuracy: 62.88%
Epoch (8/70),Train Loss: 1.0226, Val Loss: 1.3134, Val Accuracy: 60.98%
Epoch (9/70),Train Loss: 0.9976, Val Loss: 0.8035, Val Accuracy: 72.90%
Epoch (10/70),Train Loss: 0.9940, Val Loss: 0.7988, Val Accuracy: 73.14%
Epoch (11/70),Train Loss: 0.9752, Val Loss: 1.0421, Val Accuracy: 65.08%
Epoch (12/70),Train Loss: 0.9643, Val Loss: 0.9821, Val Accuracy: 66.74%
Epoch (13/70),Train Loss: 0.9578, Val Loss: 0.9186, Val Accuracy: 69.84%
Epoch (14/70),Train Loss: 0.9456, Val Loss: 0.9485, Val Accu

In [43]:
net = MyResnet()
checkpoint = torch.load("trained_model.pth")
net.load_state_dict(checkpoint["model_state"])

/tmp/ipykernel_87772/749520413.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("trained_model.pth")


<All keys matched successfully>

In [44]:
correct = 0
total = 0

net = net.to(device)
net.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 93.98%


In [45]:
stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['data/tests/ship.jpg', 'data/tests/cat.jpg', 'data/tests/dog.jpg']

images = [load_image(path).to(device) for path in image_paths]

net.eval()
print("\n--- Testowanie własnych obrazów ---")
with torch.no_grad():
    for i, image in enumerate(images):
        outputs = net(image)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
        
        class_name = class_names[predicted.item()]
        conf_percent = confidence.item() * 100
        
        print(f'Image: {image_paths[i]}, Predicted class: {class_name} ({conf_percent:.2f}%)')


--- Testowanie własnych obrazów ---
Image: data/tests/ship.jpg, Predicted class: ship (97.66%)
Image: data/tests/cat.jpg, Predicted class: cat (99.99%)
Image: data/tests/dog.jpg, Predicted class: dog (99.90%)
